In [ ]:
# Step 1 - Environment Setup

# First, we upgrade pip to ensure it can handle complex dependency resolution
#   without conflict errors (common in colab/kaggle environments).
!pip install --upgrade pip

# Installing Pytorch (The Deep Learning Engine):
#   We strictly pin Torch 2.4.0 + CUDA 11.8 to ensure compatibility with Nerfstudio.
#   --index-url: Tells pip to download the specific GPU-enabled wheels 
#   directly from PyTorch, rather than the standard CPU versions found on PyPI.
#   Without this, the code would run on the CPU (taking days).
!pip install torch==2.4.0+cu118 torchvision==0.19.0+cu118 --index-url https://download.pytorch.org/whl/cu118

# Installing Nerfstudio:
#   Installs the main framework along with compiled 
#   CUDA extensions (like gsplat) required for rendering Gaussian Splats.
!pip install nerfstudio

In [ ]:
# Step 2 - Input Preparation

import os
import shutil

# Kaggle mounts uploaded datasets in "/kaggle/input", which is a READ-ONLY directory.
#   We need to find the video and move it to "/kaggle/working" (WRITEABLE) so we can 
#   process it without permission errors.
search_path = "/kaggle/input"
found_video = None

# We use os.walk() to look through all folders in the input directory.
#   It works regardless of what the user named their video file or which subfolder it's in.
for root, dirs, files in os.walk(search_path):
    for file in files:
        # Check for common video container formats
        if file.lower().endswith(('.mp4', '.mov', '.avi', '.mkv')):
            found_video = os.path.join(root, file)
            break
    if found_video:
        break

# We copy and rename the file to "input_video.mp4".
if found_video:
    destination = "/kaggle/working/input_video.mp4"
    shutil.copy(found_video, destination)
    print(f"Video copied to: {destination}")
else:
    print("No video found.")

In [ ]:
# Step 3 - Preprocesing (Structure-from-Motion)

# COLMAP: The engine for Structure-from-Motion (calculates camera poses).
# FFmpeg: Required to extract individual frames from the input video file.
# Xvfb (X Virtual Framebuffer): Essential for "headless" cloud environments like Kaggle.
# It simulates a monitor so COLMAP's GUI-based features don't crash the session.
!apt-get update && apt-get install -y colmap ffmpeg xvfb

# 1. FFmpeg slices video -> images.
# 2. COLMAP extracts features (SIFT) and matches them across frames.
# 3. COLMAP calculates the sparse 3D point cloud and camera intrinsics/extrinsics.

# xvfb-run: Wraps the command in a virtual display server to prevent "no display" errors.
# ns-process-data: Nerfstudio's wrapper to turn raw video into training data.
# --colmap-model-path colmap: Ensures it points to the installed binary
# --gpu: forces feature (SIFT) extraction to run on the T4 GPU (10x faster than CPU).
!xvfb-run -a -s "-screen 0 1024x768x24" ns-process-data video \ 
    --data /kaggle/working/input_video.mp4 \
    --output-dir processed_data \
    --colmap-model-path colmap \ 
    --gpu 

In [ ]:
# Step 4 - Gaussian Splatting Model Training

# MAX_JOBS=5: Forces the system to use fewer parallel workers
#   for Kaggle to prevent crashing the 30GB RAM limit during dataloading.

# We use 'splatfacto', Nerfstudio's implementation of 3D Gaussian Splatting.

# --vis tensorboard: Logs training metrics (Loss, PSNR) to TensorBoard instead of 
#   launching the interactive viewer. Best for headless cloud environments.

# --max-num-iterations 15000: A balance between quality and speed.

# --pipeline.model.cull_alpha_thresh 0.01: Aggressively deletes points that are 
#   almost transparent (alpha < 1%). This reduces file size and removes "haze".

# --pipeline.model.stop_split_at 10000: Stops densifying (adding new points) 
#   after step 10k. The final 5k steps focus only on refining colors/positions.
#   This prevents the file size from exploding with unnecessary noise.

# --viewer.quit-on-train-completion True:
#    Tells the process to automatically shut down after 15,000 steps. 
!export MAX_JOBS=5 && ns-train splatfacto --data processed_data \
    --vis tensorboard \
    --max-num-iterations 15000 \
    --pipeline.model.cull_alpha_thresh 0.01 \
    --pipeline.model.stop_split_at 10000 \
    --viewer.quit-on-train-completion True

In [ ]:
# Step 5 - Export and Convert Model

import os
import glob
import shutil

# After training, Nerfstudio saves the model checkpoint and config file
#   in a timestamped folder inside "outputs/". We need to find it to export.
config_files = glob.glob("outputs/processed_data/splatfacto/*/config.yml")

if not config_files:
    print("No config file found.")
else:
    # If multiple training runs exist, we pick the latest one (alphabetically last).
    latest_config = sorted(config_files)[-1]

    # ns-export gaussian-splat: Converts Nerfstudio's internal checkpoint format
    #   into a standard .ply (Polygon File Format) point cloud.
    !ns-export gaussian-splat --load-config {latest_config} --output-dir exports/splat

    # The raw .ply format is too heavy for standard web loading.
    #   We use antimatter15's convert.py script to transform the .ply into
    #   a .splat file, which is optimized for real-time WebGL rendering
    #   inside our application (via React-Three-Fiber / Splat library).
    if not os.path.exists("convert.py"):
        !wget -q https://raw.githubusercontent.com/antimatter15/splat/main/convert.py

    input_ply = "exports/splat/splat.ply"

    if os.path.exists(input_ply):
        
        # The convert.py script only takes the input file as an argument.
        #   It automatically appends ".splat" to create the output filename.
        !python convert.py {input_ply}
        
        generated_auto_name = "output.splat"
        final_destination = "/kaggle/working/scene.splat"
        
        # Move the generated file to the working directory root for easy download.
        if os.path.exists(generated_auto_name):
            if os.path.exists(final_destination):
                os.remove(final_destination)
            
            shutil.move(generated_auto_name, final_destination)
            print(f"File created: {final_destination}")
        else:
            print(f"Expected file '{generated_auto_name}' is missing.")
            
    else:
        print("PLY file not found.")